In [2]:
!pip install ultralytics easyocr opencv-python-headless numpy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 37.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.9/2.9 MB 97.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.7/180.7 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 963.8/963.8 kB 62.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 292.1/292.1 kB 23.2 MB/s eta 0:00:00


In [3]:
from ultralytics import YOLO
import easyocr

model=YOLO("license_plate_best.pt")

text_reader=easyocr.Reader(lang_list=['en'],gpu=True)

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Progress: |██████████████████████████████████████████████████| 100.0% Complete

Progress: |██████████████████████████████████████████████████| 100.0% Complete

In [23]:
import re
import numpy

licence_plate_pattren=re.compile(r"[A-Z]{2}[0-9]{2}[A-Z]{3}")

print(licence_plate_pattren)

re.compile('[A-Z]{2}[0-9]{2}[A-Z]{3}')


In [24]:
input_vedio="cars.mp4"
output_vedio="cars_prediction.mp4"

In [25]:
import cv2

cap=cv2.VideoCapture(input_vedio)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')

out=cv2.VideoWriter(filename=output_vedio,fourcc=fourcc,fps=cap.get(cv2.CAP_PROP_FPS),frameSize=(int(cap.get(3)),int(cap.get(4))))

In [26]:
conf_threshold=0.4

In [27]:
def checking_and_correcting_ocr_text(ocr_text):

  mapping_num_to_char={'0': 'O', '1': 'I', '2': 'Z', '3': 'B', '4': 'A', '5': 'S', '6': 'G', '7': 'T', '8': 'B', '9': 'P'}
  mapping_char_to_num={'O': '0', 'I': '1', 'Z': '2', 'B': '3', 'A': '4', 'S': '5', 'G': '6', 'T': '7', 'P': '9', 'Q': '0'}

  modified_text=[]

  ocr_text=''.join(ocr_text.upper().split())

  if len(ocr_text)!=7:
    return ''

  for index,sym in enumerate(ocr_text):

    if (index>=2) and (index<=3):

      if sym.isalpha():
        modified_text.append(mapping_char_to_num.get(sym,sym))
      else:
        modified_text.append(sym)

    else:

      if sym.isdigit():
        modified_text.append(mapping_num_to_char.get(sym,sym))
      else:
        modified_text.append(sym)

  if len(modified_text):

    return(''.join(modified_text))

  return ''


In [33]:
def recognize_text(plate_cropped):

  if plate_cropped.size==0:
    return ''

  gray_image=cv2.cvtColor(plate_cropped,cv2.COLOR_BGR2GRAY)

  thresh_gray_image=cv2.threshold(gray_image, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)[1]

  h, w = thresh_gray_image.shape
  dsize = (int(w*3), int(h*3))

  scaled_image=cv2.resize(thresh_gray_image, dsize=dsize, interpolation=cv2.INTER_CUBIC)

  try:

    ocr_text=text_reader.readtext(scaled_image,detail=0,allowlist=r"[A-Z0-9]")

    licence_text=checking_and_correcting_ocr_text(ocr_text[0])

    if licence_text and licence_plate_pattren.match(licence_text):
      return licence_text

  except:
    pass

  return ''

In [29]:
def creating_box_id(x1,y1,x2,y2):

  return f"{int(x1/10)}_{int(y1/10)}_{int(x2/10)}_{int(y2/10)}"

In [30]:
from collections import defaultdict,deque

plate_history=defaultdict(lambda: deque(maxlen=10))
plate_final={}

In [31]:
def stabilizing_plate_number(box_id,licence_number):

  if licence_number:

    plate_history[box_id].append(licence_number)

    most_repeating=max(set(plate_history[box_id]),key=plate_history[box_id].count)

    plate_final[box_id]=most_repeating

    return plate_final[box_id]

  return ''


In [ ]:
while cap.isOpened():

    ret,frame=cap.read()

    if not ret:
        break

    output=model(frame)

    for i in output:

        for b in i.boxes:

            conf_predicted=float(b.conf.cpu().numpy()[0])

            if conf_predicted<conf_threshold:
                continue

            x1,y1,x2,y2=map(int,b.xyxy[0].cpu().numpy())

            plate_cropped=frame[y1:y2,x1:x2]

            licence_number=recognize_text(plate_cropped)

            box_id=creating_box_id(x1,y1,x2,y2)

            stable_plate_number=stabilizing_plate_number(box_id,licence_number)

            cv2.rectangle(img=frame,pt1=(x1,y1),pt2=(x2,y2),color=(0,255,0),thickness=2)

            if plate_cropped.size>0:

                overlay_h, overlay_w=150,400
                plate_resized=cv2.resize(plate_cropped,(overlay_w,overlay_h))
                oy1=max(0,y1-overlay_h-40)
                ox1=x1
                oy2, ox2 = oy1 + overlay_h, ox1 + overlay_w
                if oy2 <= frame.shape[0] and ox2 <= frame.shape[1]:
                    frame[oy1:oy2, ox1:ox2] = plate_resized

            if stable_plate_number:

              cv2.putText(frame,stable_plate_number,(x1,y1-20),cv2.FONT_HERSHEY_SIMPLEX,3,(0,0,0),6)

              cv2.putText(frame,stable_plate_number,(x1,y1-20),cv2.FONT_HERSHEY_SIMPLEX,3,(255,255,255),3)


    out.write(frame)

cap.release()
out.release()

print("Output Vedio Generated Succesfully")


0: 384x640 2 License_Plates, 124.5ms
Speed: 5.6ms preprocess, 124.5ms inference, 0.9ms postprocess per image at shape (1, 3, 384, 640)


/usr/local/lib/python3.12/dist-packages/torch/utils/data/dataloader.py:666: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)



0: 384x640 2 License_Plates, 136.7ms
Speed: 4.2ms preprocess, 136.7ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 140.0ms
Speed: 4.4ms preprocess, 140.0ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 136.8ms
Speed: 6.6ms preprocess, 136.8ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 License_Plates, 122.9ms
Speed: 3.9ms preprocess, 122.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 214.0ms
Speed: 3.8ms preprocess, 214.0ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 141.9ms
Speed: 6.8ms preprocess, 141.9ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 3 License_Plates, 130.3ms
Speed: 5.2ms preprocess, 130.3ms inference, 0.8ms postprocess per image at shape (1, 3, 384, 640)

0: 384x640 2 License_Plates, 124.6ms
Speed: 3.9